### Project 5: Relational Database
### Author: Chad Bell
### Date: 3-10-2023
### Description: Recreate the PREVAIL database

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import psycopg2

In [6]:
# Establish a connection with the postgres database
con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="example_password")
# Create a cursor object
cur = con.cursor()

In [68]:
# con.commit()
con.rollback()

In [83]:
# Create loan types table
cur.execute("DROP TABLE IF EXISTS loan_types CASCADE;")
cur.execute("""CREATE TABLE loan_types (
    id TEXT PRIMARY KEY NOT NULL,
    name TEXT,
    interest_rate NUMERIC,
    loan_term NUMERIC,
    pay_frequency NUMERIC,
    maturity_bonus NUMERIC,
    referral_rate NUMERIC,
    type_notes TEXT
    );"""
            )

# Create loans table
cur.execute("DROP TABLE IF EXISTS loans CASCADE;")
cur.execute("""CREATE TABLE loans (
    id TEXT PRIMARY KEY NOT NULL,
    initial_start_date DATE,
    qualified TEXT,
    ira_num TEXT,
    loan_notes TEXT,
    partner_paid DATE,
    promissory_note BYTEA,
    loan_type_id TEXT REFERENCES loan_types (id)
  );"""
            )
# Create loan instances table
cur.execute("DROP TABLE IF EXISTS loan_instances CASCADE;")
cur.execute("""CREATE TABLE loan_instances (
    id TEXT PRIMARY KEY NOT NULL,
    amount NUMERIC,
    stage TEXT,
    app_submitted TIMESTAMP,
    funds_wired TIMESTAMP,
    start_date DATE,
    date_paid TIMESTAMP,
    loan_num NUMERIC,
    instance_notes TEXT,
    loan_id TEXT REFERENCES loans (id)
  );"""
            )

# Create clients table
cur.execute("DROP TABLE IF EXISTS clients CASCADE;")
cur.execute("""CREATE TABLE clients (
    id TEXT PRIMARY KEY NOT NULL,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    zip TEXT,
    client_notes TEXT
    );"""
            )
# Create referring partners table
cur.execute("DROP TABLE IF EXISTS referring_partners CASCADE;")
cur.execute("""CREATE TABLE referring_partners (
    id TEXT PRIMARY KEY NOT NULL,
    name TEXT,
    email TEXT,
    referral_rate NUMERIC,
    partner_notes TEXT
  );""")

# Many-to-many relationship between clients and referring partners
cur.execute("DROP TABLE IF EXISTS client_referring_partner CASCADE;")
cur.execute("""CREATE TABLE client_referring_partner (
    client_id TEXT REFERENCES clients (id),
    referring_partner_id TEXT REFERENCES referring_partners (id),
    PRIMARY KEY (client_id, referring_partner_id)
  );"""
            )

# Many-to-many relationship between loans and clients
cur.execute("DROP TABLE IF EXISTS loan_client CASCADE;")
cur.execute("""CREATE TABLE loan_client (
    loan_id TEXT REFERENCES loans (id),
    client_id TEXT REFERENCES clients (id),
    PRIMARY KEY (loan_id, client_id)
  );"""
            )

con.commit()


In [65]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema='public';")
cur.fetchall()

[('postgres',
  'public',
  'clients',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'client_referring_partner',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'referring_partners',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loan_client',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loan_types',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loans',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loan_instances',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None)]

In [84]:
def create_loan_type(id, name, interest_rate, loan_term, pay_frequency, maturity_bonus, referral_rate, type_notes):
    """Create a new loan type"""
    cur.execute("""INSERT INTO loan_types (id, name, interest_rate, loan_term, pay_frequency, maturity_bonus, referral_rate, type_notes)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """, 
                (id, name, interest_rate, loan_term, pay_frequency, maturity_bonus, referral_rate, type_notes))
    con.commit()
    
def create_loan(id, initial_start_date, qualified, ira_num, loan_notes, partner_paid, promissory_note, loan_type_id):
    """Create a new loan"""
    cur.execute("""INSERT INTO loans (id, initial_start_date, qualified, ira_num, loan_notes, partner_paid, promissory_note, loan_type_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """, 
                (id, initial_start_date, qualified, ira_num, loan_notes, partner_paid, promissory_note, loan_type_id))
    con.commit()

def create_loan_instance(id, amount, stage, app_submitted, funds_wired, start_date, date_paid, loan_num, instance_notes, loan_id):
    """Create a new loan instance"""
    cur.execute("""INSERT INTO loan_instances (id, amount, stage, app_submitted, funds_wired, start_date, date_paid, loan_num, instance_notes, loan_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, 
                (id, amount, stage, app_submitted, funds_wired, start_date, date_paid, loan_num, instance_notes, loan_id))
    con.commit()

def create_client(id, first_name, last_name, email, phone, address, city, state, zip, client_notes):
    """Create a new client"""
    cur.execute("""INSERT INTO clients (id, first_name, last_name, email, phone, address, city, state, zip, client_notes)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, 
                (id, first_name, last_name, email, phone, address, city, state, zip, client_notes))
    con.commit()

def create_referring_partner(id, name, email, referral_rate, partner_notes):
    """Create a new referring partner"""
    cur.execute("""INSERT INTO referring_partners (id, name, email, referral_rate, partner_notes)
                VALUES (%s, %s, %s, %s, %s);
                """, 
                (id, name, email, referral_rate, partner_notes))
    con.commit()

def create_client_referring_partner(client_id, referring_partner_id):
    """Create a new client referring partner relationship"""
    cur.execute("""INSERT INTO client_referring_partner (client_id, referring_partner_id)
                VALUES (%s, %s);
                """, 
                (client_id, referring_partner_id))
    con.commit()

def create_loan_client(loan_id, client_id):
    """Create a new loan client relationship"""
    cur.execute("""INSERT INTO loan_client (loan_id, client_id)
                VALUES (%s, %s);
                """, 
                (loan_id, client_id))
    con.commit()

def get_loan_type(id):
    """Get a loan type by id"""
    cur.execute("""SELECT * FROM loan_types WHERE id = %s;""", (id,))
    return cur.fetchone()

def get_loan(id):
    """Get a loan by id"""
    cur.execute("""SELECT * FROM loans WHERE id = %s;""", (id,))
    return cur.fetchone()

def get_loan_instance(id):
    """Get a loan instance by id"""
    cur.execute("""SELECT * FROM loan_instances WHERE id = %s;""", (id,))
    return cur.fetchone()


In [81]:
con.rollback()

In [85]:
# Create loan types
for loan_type in pd.DataFrame(pd.read_csv('loan_types.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_loan_type(**loan_type)
# Create loans
for loan in pd.DataFrame(pd.read_csv('loans.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_loan(**loan)


# Create loan instances
for loan_instance in pd.DataFrame(pd.read_csv('loan_instances.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_loan_instance(**loan_instance)
# Create clients
for client in pd.DataFrame(pd.read_csv('clients.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_client(**client)
# Create referring partners
for referring_partner in pd.DataFrame(pd.read_csv('referring_partners.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_referring_partner(**referring_partner)
# Create client referring partner relationships
for client_referring_partner in pd.DataFrame(pd.read_csv('client_referring_partner.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_client_referring_partner(**client_referring_partner)
# Create loan client relationships
for loan_client in pd.DataFrame(pd.read_csv('loan_client.csv')).replace(np.nan, None, regex=True).to_dict('records'):
    create_loan_client(**loan_client)

con.commit()

In [94]:
# Get all of the loan types and count the number of loans for each type
cur.execute("""SELECT name, interest_rate, loan_term, ROUND(interest_rate/loan_term*12, 2) rate, COUNT(*) FROM loans
JOIN loan_types ON loans.loan_type_id = loan_types.id
GROUP BY name, interest_rate, loan_term;
""")
df = pd.DataFrame(cur.fetchall())
df.rename(columns={0: 'name', 1: 'interest_rate', 2: 'loan_term', 3: 'rate', 4: 'count'}, inplace=True)

In [97]:
import plotly.graph_objs as go

x = df['name']
y = df['count']

# Create traces
trace1 = go.Bar(x=x, y=y, name='Count')

# Create layout
layout = go.Layout(title='Loan Counts by Type', xaxis=dict(title='Types'), yaxis=dict(title='Count'))

# Create figure and plot
fig = go.Figure(data=[trace1], layout=layout)
fig.show()